In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.4-py2.py3-none-any.whl size=265375577 sha256=e5785ad3c3381e79391ad34776f18773c0a7947ef391be437c6c5074f65ece51
  Stored in directory: /root/.cache/pip/wheels/4d/a6/47/8bfeb1026fd65cb8630beb74d8e3bec844f572cf4f336fdd56
Successfully built h2o


In [ ]:
%%writefile main.py

import streamlit as st
import h2o
import pandas as pd
from io import StringIO, BytesIO
import atexit
import base64
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Инициализация h2o
h2o.init()

# Заголовок приложения
st.title('Интерактивное обучение модели')

# Левое меню с кнопками
menu_selection = st.sidebar.radio('Навигация', ['Главная', '1. Загрузка датасета', '2. Создание модели', '3. Обучение и предсказание'])

# Функция для загрузки и обработки датасета
def load_dataset(file):
    # Проверка размера файла
    if file.size > 500 * 1024 * 1024:
        st.error("Размер файла превышает 500 МБ")
        return None

    # Определение типа файла и его загрузка
    try:
        if file.type == 'text/csv':
            df = pd.read_csv(file)
        elif file.type == 'text/plain':
            df = pd.read_csv(file, delimiter='\t')
        else:
            df = pd.read_excel(file)

        # Конвертация DataFrame в h2o Frame
        h2o_df = h2o.H2OFrame(df)
        st.success("Датасет успешно загружен и конвертирован в h2o Frame")
        return h2o_df
    except Exception as e:
        st.error(f"Ошибка при загрузке файла: {e}")
        return None

# Инициализация пустого списка слоев и данных в session state
if 'layers' not in st.session_state:
    st.session_state.layers = []

if 'x' not in st.session_state:
    st.session_state.x = []

if 'y' not in st.session_state:
    st.session_state.y = None

if 'model' not in st.session_state:
    st.session_state.model = None

if 'train_data' not in st.session_state:
    st.session_state.train_data = None

# В зависимости от выбранного пункта меню выводим соответствующую информацию или страницу
if menu_selection == 'Главная':
    st.write('Добро пожаловать!')
    st.write('Инструкция для выполнения:')
    st.write('1. Перейдите на страницу загрузки датасета и загрузите свой датасет. Затем выберите желаемую метку.')
    st.write('2. Создайте модель: определите количество слоёв, нейронов и регуляризацию')
    st.write('3. Обучите модель. Если вы довольны результатом обучения, загрузите тестовую выборку. Приложение вставит предугаданные метки и позволит вам сохранить результат')

elif menu_selection == '1. Загрузка датасета':
    st.header('Загрузка датасета')

    # Загрузка файла
    uploaded_file = st.file_uploader("Загрузите датасет", type=['csv', 'txt', 'xls', 'xlsx'])

    if uploaded_file:
        h2o_df = load_dataset(uploaded_file)

        if h2o_df:
            st.write("Первые 5 строк датасета:")
            st.write(h2o_df.head(5))

            # Выбор целевого элемента
            target = st.selectbox("Выберите целевую переменную", h2o_df.columns)

            if target:
                # Определение предикторов и целевой переменной
                st.session_state.x = h2o_df.columns
                st.session_state.y = target
                st.session_state.x.remove(st.session_state.y)
                st.session_state.train_data = h2o_df

                st.write(f"Целевая переменная: {st.session_state.y}")
                st.write(f"Предикторы: {st.session_state.x}")
elif menu_selection == '2. Создание модели':
    st.header('Создание модели')

    # Добавление нового слоя
    if st.button('Добавить слой'):
        st.session_state.layers.append({'type': 'Dense', 'neurons': 1, 'activation': 'relu'})

    # Отображение текущих слоев
    for i, layer in enumerate(st.session_state.layers):
        st.subheader(f'Слой {i + 1}')

        # Тип слоя (пока только Dense)
        layer['type'] = st.selectbox(f'Тип слоя {i + 1}', ['Dense'], key=f'type_{i}')

        # Количество нейронов
        layer['neurons'] = st.number_input(f'Количество нейронов в слое {i + 1}', min_value=1, value=layer['neurons'], key=f'neurons_{i}')

        # Активация
        layer['activation'] = st.selectbox(f'Функция активации для слоя {i + 1}', ['relu', 'sigmoid', 'tanh', 'softmax'], key=f'activation_{i}')

        # Удаление слоя
        if st.button(f'Удалить слой {i + 1}'):
            st.session_state.layers.pop(i)
            st.experimental_rerun()

    # Создание модели
    if st.button('Создать модель'):
        model = Sequential()
        for layer in st.session_state.layers:
            model.add(Dense(layer['neurons'], activation=layer['activation']))

        try:
            model.build(input_shape=(None, len(st.session_state.x)))  # Используем предикторы из session_state
            st.session_state.model = model  # Сохраняем модель в session_state
            st.success('Модель успешно создана!')
            st.write(model.summary())
        except Exception as e:
            st.error(f'Ошибка при создании модели: {e}')
elif menu_selection == '3. Обучение и предсказание':
    st.header('Обучение и предсказание')

    # Проверка, создана ли модель
    if st.session_state.model is None:
        st.error('Сначала создайте модель на странице "2. Создание модели"')
    elif st.session_state.train_data is None:
        st.error('Сначала загрузите датасет на странице "1. Загрузка датасета"')
    else:
        # Параметры обучения
        epochs = st.number_input('Количество эпох', min_value=1, value=10)
        batch_size = st.number_input('Размер батча', min_value=1, value=32)
        learning_rate = st.number_input('Скорость обучения', min_value=0.0001, step=0.0001, value=0.001)

        # Обучение модели
        if st.button('Обучить модель'):
            try:
                # Подготовка данных для обучения
                train_data = st.session_state.train_data.as_data_frame()

                # Удаление строк с пустыми значениями
                train_data.dropna(inplace=True)

                # Преобразование категориальных переменных в числовые
                train_data = pd.get_dummies(train_data)

                X_train = train_data[st.session_state.x].values
                y_train = train_data[st.session_state.y].values

                # Преобразование типов данных в float
                X_train = X_train.astype(float)
                y_train = y_train.astype(float)

                # Компиляция и обучение модели
                st.session_state.model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
                history = st.session_state.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

                # Отображение результатов обучения
                st.write('Результаты обучения:')
                st.line_chart(history.history['loss'])
                st.line_chart(history.history['val_loss'])

                st.success('Модель успешно обучена!')
            except Exception as e:
                st.error(f'Ошибка при обучении модели: {e}')

        # Загрузка тестовой выборки
        test_file = st.file_uploader("Загрузите тестовую выборку", type=['csv', 'txt', 'xls', 'xlsx'])

        if test_file:
            try:
                # Загрузка и обработка тестовой выборки
                if test_file.type == 'text/csv':
                    test_df = pd.read_csv(test_file)
                elif test_file.type == 'text/plain':
                    test_df = pd.read_csv(test_file, delimiter='\t')
                else:
                    test_df = pd.read_excel(test_file)

                # Преобразование категориальных переменных в числовые
                test_df = pd.get_dummies(test_df)

                # Убедиться, что тестовые данные имеют те же столбцы, что и тренировочные данные
                missing_cols = set(train_data.columns) - set(test_df.columns)
                for col in missing_cols:
                    test_df[col] = 0

                X_test = test_df[st.session_state.x].values

                # Преобразование типов данных в float
                X_test = X_test.astype(float)

                # Предсказание
                predictions = st.session_state.model.predict(X_test)
                test_df[st.session_state.y] = predictions

                st.write("Результаты предсказания:")
                st.write(test_df)

                # Скачивание результатов предсказания
                csv = test_df.to_csv(index=False)
                b64 = base64.b64encode(csv.encode()).decode()  # some strings <-> bytes conversions necessary here
                href = f'<a href="data:file/csv;base64,{b64}" download="predictions.csv">Скачать результаты предсказания</a>'
                st.markdown(href, unsafe_allow_html=True)
            except Exception as e:
                st.error(f'Ошибка при обработке тестовой выборки: {e}')

# Завершение работы h2o
atexit.register(h2o.shutdown)


Writing main.py


In [ ]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

Forwarding HTTP traffic from https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net
HTTP request from 90.151.81.50 to https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net/
HTTP request from 90.151.81.50 to https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net/static/css/main.3aaaea00.css
HTTP request from 90.151.81.50 to https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net/static/media/SourceSansPro-SemiBold.abed79cd0df1827e18cf.woff2
HTTP request from 90.151.81.50 to https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net/static/media/SourceSansPro-Bold.118dea98980e20a81ced.woff2
HTTP request from 90.151.81.50 to https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net/static/js/main.7994a814.js
HTTP request from 90.151.81.50 to https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net/static/media/SourceSansPro-Regular.0d69e5ff5e92ac64a0c9.woff2
HTTP request from 90.151.81.50 to https://977f951bf12a153c9227bf1a2a10d1cf.serveo.net/_stcore/health
HTTP request from 90.151.81.50 to https://977f951bf12a153c922